# Data Import and Setup

In [ ]:
import pandas as pd

In [ ]:
# Loading the file
file_path = '/Users/tejuskhandelwal/Desktop/UGP/GWQ_2010-2018.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')
print(df.head)

<bound method NDFrame.head of                 SITE_ID WRIS ID   LATITUDE  LONGITUDE SITE_TYPE  \
0      W115159092441001  W23881  11.866389  92.736111  Dug Well   
1                   New  W38814  12.186944  92.791667  Dug Well   
2      W123014092513301  W24095  12.503889  92.859167  Dug Well   
3      W123129092581301  W24112  12.524722  92.970278  Dug Well   
4      W123030092573801  W24098  12.508333  92.960556  Dug Well   
...                 ...     ...        ...        ...       ...   
85192               New  W40943      22.51      87.33  Dug Well   
85193  W220857087092501  W10716      22.18      87.16  Dug Well   
85194               New  W40944      22.58      87.52  Dug Well   
85195               New  W40945      22.47      87.02  Dug Well   
85196               New  W40946      22.61      86.81  Dug Well   

                STATE_NAME  DISTRICT_NAME     TAHSIL_NAME       BLOCK_NAME  \
0      Andaman And Nicobar  A & N Islands      Ferrargunj    South Andaman   
1      An

# Data Preprocessing
This section includes handling missing values, cleaning, and preparing data for analysis.

In [ ]:
# Separating important columns
columns_to_extract = ['PH', 'TH', 'CA', 'MG', 'CHLORIDE', 'SULPHATE', 'NITRATE', 'FLUORIDE', 'TDS']
new_df = df[columns_to_extract]

In [ ]:
print(new_df.head)

<bound method NDFrame.head of          PH   TH  CA      MG CHLORIDE SULPHATE NITRATE FLUORIDE      TDS
0      8.34  130  22  18.225   17.725     3.21    3.63     0.37  217.748
1      8.46  120  14  20.655   38.995    46.22    2.46     0.24  360.745
2      8.11  160  12   31.59   17.725    30.46       0     0.96  239.246
3      7.89  200  22  35.235   17.725    13.34       0     1.02   253.22
4      8.01  125  12  23.085    14.18    13.37       0     0.74  185.361
...     ...  ...  ..     ...      ...      ...     ...      ...      ...
85192  7.83   60  18   3.645   10.665        0       0        0   75.584
85193   7.8  125  38    7.29    85.32       27      19     0.05   316.16
85194  7.85  200  38  25.515    78.21       97      42     0.38  307.776
85195  8.05   20   2   3.645     7.11        0       0        0  25.2544
85196  7.71  175  58    7.29    85.32       16      45     0.34    339.2

[85197 rows x 9 columns]>


In [ ]:
# Handling non numeric types
new_df.replace(['ND', 'BDL'], 0, inplace=True)

/var/folders/s8/md_3hxqs0455zql3_p9ydl4w0000gn/T/ipykernel_68324/2315555786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.replace(['ND', 'BDL'], 0, inplace=True)


In [ ]:
print((new_df == 'ND').sum())

PH          0
TH          0
CA          0
MG          0
CHLORIDE    0
SULPHATE    0
NITRATE     0
FLUORIDE    0
TDS         0
dtype: int64


In [ ]:
print((new_df == 'BDL').sum())

PH          0
TH          0
CA          0
MG          0
CHLORIDE    0
SULPHATE    0
NITRATE     0
FLUORIDE    0
TDS         0
dtype: int64


In [ ]:
print(new_df.isnull().sum())

PH              4
TH           3417
CA           3137
MG           3148
CHLORIDE       12
SULPHATE     3137
NITRATE      4141
FLUORIDE     1205
TDS         63720
dtype: int64


In [ ]:
# Converting data to numeric
new_df = new_df.apply(pd.to_numeric, errors='coerce')

In [ ]:
print(new_df.isnull().sum())

PH             21
TH           3422
CA           4678
MG           4039
CHLORIDE       17
SULPHATE     3240
NITRATE      4280
FLUORIDE     1373
TDS         64071
dtype: int64


In [ ]:
print((new_df == 0).sum())

PH           1507
TH           5714
CA           5671
MG           5711
CHLORIDE     3637
SULPHATE    13958
NITRATE     14987
FLUORIDE     6929
TDS          7663
dtype: int64


In [ ]:
new_df.dropna(inplace=True)

In [ ]:
# print(new_df.head)
print(new_df.shape[0])

19110


In [ ]:
# These are the standards as defined by Bureau of Indian Standards
bis_standards = {
    'PH': (6.5, 8.5),        # Acceptable range for pH
    'TH': (0, 200),          # Total Hardness in mg/L
    'CA': (0, 75),           # Calcium in mg/L
    'MG': (0, 30),           # Magnesium in mg/L
    'CHLORIDE': (0, 250),    # Chloride in mg/L
    'SULPHATE': (0, 200),    # Sulphate in mg/L
    'NITRATE': (0, 45),      # Nitrate in mg/L
    'FLUORIDE': (0, 1),    # Fluoride in mg/L
    'TDS': (0, 500)          # Total Dissolved Solids in mg/L
}

In [ ]:
# Based on whether the data is within the limits we set if water is healthy or unhealthy
def check_health(row):
    for param, (low, high) in bis_standards.items():
        if pd.isna(row[param]) or not (low <= row[param] <= high):
            return 0 
    return 1 

In [ ]:
print((new_df == 0).sum())

PH                   2
TH                 175
CA                  85
MG                 126
CHLORIDE             4
SULPHATE           739
NITRATE           3193
FLUORIDE          1929
TDS               6893
Health_Status    14102
dtype: int64


In [ ]:
new_df['Health_Status'] = new_df.apply(check_health, axis=1)

In [ ]:
print(new_df.head)

<bound method NDFrame.head of          PH     TH    CA      MG  CHLORIDE  SULPHATE  NITRATE  FLUORIDE  \
0      8.34  130.0  22.0  18.225    17.725      3.21     3.63      0.37   
1      8.46  120.0  14.0  20.655    38.995     46.22     2.46      0.24   
2      8.11  160.0  12.0  31.590    17.725     30.46     0.00      0.96   
3      7.89  200.0  22.0  35.235    17.725     13.34     0.00      1.02   
4      8.01  125.0  12.0  23.085    14.180     13.37     0.00      0.74   
...     ...    ...   ...     ...       ...       ...      ...       ...   
85192  7.83   60.0  18.0   3.645    10.665      0.00     0.00      0.00   
85193  7.80  125.0  38.0   7.290    85.320     27.00    19.00      0.05   
85194  7.85  200.0  38.0  25.515    78.210     97.00    42.00      0.38   
85195  8.05   20.0   2.0   3.645     7.110      0.00     0.00      0.00   
85196  7.71  175.0  58.0   7.290    85.320     16.00    45.00      0.34   

            TDS  Health_Status  
0      217.7480              1  
1  

In [ ]:
print((new_df['Health_Status']==1).sum())

5008


In [ ]:
print((new_df['Health_Status']==0).sum())

14102


In [ ]:
# Exporting
output_file_path = '/Users/tejuskhandelwal/Desktop/UGP/training_data.csv' 
new_df.to_csv(output_file_path, index=False)